In [1]:
import requests      # Для запросов по API
import json          # Для обработки полученных результатов
import time          # Для задержки между запросами
import os            # Для работы с файлами
import re
import pandas as pd  # Для формирования датафрейма с результатами
from bs4 import BeautifulSoup

In [2]:


def get_vacancy(query = 'ML'):
    params = {
        'text': query,         # Поиск текста
        # 'area': area,         # Поиск в зоне
        # 'page': page,         # Номер страницы
        'per_page': 100       # Кол-во вакансий на 1 странице
    }   
    req = requests.get('https://api.hh.ru/vacancies', params)
    data = json.loads(req.content.decode())
    req.close()
    return data

def get_skills(vacancy):
    req = requests.get(vacancy['url'])
    data = json.loads(req.content.decode())
    req.close()
    result = []
    for i in data['key_skills']:
        result.append(i['name'])
    return result

def get_desc(vacancy):
    req = requests.get(vacancy['url'])
    data = json.loads(req.content.decode())
    req.close()
    soup = BeautifulSoup(data['description'], 'html.parser')

    # Извлекаем текст из HTML
    text = soup.get_text()
    return text

def get_requirements(vacancy):
    text1 = vacancy['snippet']['requirement']
    text2 = vacancy['snippet']['responsibility']
    pattern = r'( и т\.?\s?д\.?| и др\s?\.?|и\/или|или| и )'
    cleaned_text1 = re.sub(pattern, ' ', text1, flags=re.IGNORECASE)
    cleaned_text2 = re.sub(pattern, ' ', text2, flags=re.IGNORECASE)
    pattern = r'(\.\.\.\n\.\.\.)'
    cleaned_text1 = re.sub(pattern, ' ', cleaned_text1, flags=re.IGNORECASE)
    cleaned_text2 = re.sub(pattern, ' ', cleaned_text2, flags=re.IGNORECASE)
    pattern = r'([A-ZА-Я]([^\.]|(\. [a-zа-я]))+)\.?\ ?'
    cleaned_text1 = re.findall(pattern, cleaned_text1, flags=re.IGNORECASE)
    cleaned_text2 = re.findall(pattern, cleaned_text2, flags=re.IGNORECASE)
    return cleaned_text1 + cleaned_text2

In [3]:
from razdel import tokenize, sentenize

from navec import Navec
import slovnet

navec = Navec.load("navec_news_v1_1B_250K_300d_100q.tar")
syntax = slovnet.Syntax.load("slovnet_syntax_news_v1.tar")

syntax. navec(navec)

def seg_text(doc):
    if isinstance(doc, str):
        doc = {"text": doc}
    
    doc["tokens"] = []

    for sent in sentenize(doc["text"]):
        doc["tokens"].append([_.text for _ in tokenize(sent.text)])
    
    return doc

import nltk
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer

nltk.download("stopwords")
analyzer = MorphAnalyzer()
stop_words = stopwords.words("russian")
pos = {'NOUN', 'ADJF', 'ADJS', 'VERB', 'INFN', 'PRTF', 'PRTS'}

def extract_candidates(doc: dict, stop_words: list = stop_words, pos: set = pos) -> dict:

    res = set()
    for sent in doc["tokens"]:
        for token in sent:
            if token in stop_words or token in res:
                continue

            parsed = analyzer.parse(token) [0]

            if parsed.tag.POS not in pos:
                continue

            res.add(token)
    doc["candidates"] = res
    return doc

def syntax_collocations(doc: dict, syntax: slovnet.api.Syntax = syntax) -> dict:
    syntax_colloc = []
    for sent in doc["tokens"]:

        syntax_markup = syntax(sent)
        sent_word_id = {}
        for token in syntax_markup. tokens:
            sent_word_id[token.id] = token.text
            
        for token in syntax_markup. tokens:
            if token.head_id!='@' and token.text in doc["candidates"]:
                try:
                    syntax_colloc.append(sent_word_id[token.head_id] + ' ' + token.text)
                except :
                    pass
    doc["collocations"] = set(syntax_colloc)

    return doc

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/datalore/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:


# import spacy
import json
from pymorphy2 import MorphAnalyzer
from transliterate import translit, get_available_language_codes

from natasha import (
    Segmenter,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    MorphVocab,
    Doc
)
# Загрузка модели языка
# nlp = spacy.load("en_core_web_sm")
pattern  = r'([A-ZА-Я][a-zа-яA-ZА-Я\s\/\,]+[\:|\?|\-])\ *\n'
pattern1 = r'(требован)|(хотели.+видеть)|(ждем.+от)|(ожидаем)|(ожидания.+от)|(ждем.+)|(стек.+)'
pattern2 = r'(знан.+)|(опыт.*)|(навык.+)|(уме.+)|(.+сть)|(образов.+)|(работ.+)|(владе.+)|(понима.+)|(принци.+)|(стек.+)'\
           r'|(.+ний)|(.+чие)|(фрейм.+)|(.+ом)|(человек.+)|(уров.+)|(лет)|(рекл.+)|(вакан.+)|(получ.+)|(пользовате.+)|'\
           r'(предостав.+)|(труд.+)|(услов.+)|(откл.+)|(час.+)|(чел)|(чу.+)|(федер.+)'
pattern3 = r'[^\w]([a-zа-яA-ZА-Я]+[^;]+)[^\n+]'
pattern4 = r"[\s\w\d]"
pattern5 = r"[\s\,\.\+\/]+"
pattern6 = r"[a-zA-Z\d\s]+"

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
# Выполняем сегментацию текста
segmenter = Segmenter()
morph = MorphAnalyzer()



def get_tags_vacancy(text):
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text()
    blocks = re.findall(pattern, text.replace('\\n', ' '), re.MULTILINE)
    blocks = [i.lower() for block in blocks for i in block.split('\n')]
    req = [block for block in blocks if re.search(pattern1, block)]
    sec = None
    blk = []
    if len(req):
        for s in req:
            for j in range(blocks.index(s) + 1, len(blocks)):
                if not re.search(pattern2, blocks[j]):
                    sec = blocks[j]
            tmp = text.lower().split(s)[1].strip().strip(':').strip()
            if sec:
                blk.append(tmp.split(sec)[0].strip().strip(':').strip())
                sec = None
            else:
                blk.append(tmp)
    else:
        blk = [text.lower().strip()]

    
    tags = []
    tags1 = []
    tags2 = []
    for block in blk:    
        res = [j.replace('\n', ' ').replace('\t', ' ').replace('/', ', ') for j in re.findall(pattern3, block)]
        for text in res:
            doc = Doc(text)
            doc.segment(segmenter)
            doc.tag_morph(morph_tagger)
            doc.parse_syntax(syntax_parser)
            for token in doc.tokens:
                tag = "".join(re.findall(pattern4, token.text, re.MULTILINE))
                # tag = token.text
                if not re.search(pattern2, token.text) and len(tag) > 0:
                    if token.pos == 'NOUN':
                        tags1.append(token.text)
                        tags1.append(translit(token.text, 'ru', reversed=True))
                    elif token.pos == 'X':
                        tags.append(token.text)
                        tags.append(translit(token.text, 'ru'))

            for word in text.split():
                if re.search(pattern6, "".join(re.findall(pattern4, word, re.MULTILINE))):
                    tags.append("".join(re.findall(pattern4, word, re.MULTILINE)))
                    tags.append(translit("".join(re.findall(pattern4, word, re.MULTILINE)), 'ru'))


            doc1 = seg_text(text)
            doc1 = extract_candidates(doc1)
            doc1 = syntax_collocations(doc1)
            for tag in doc1["collocations"]:
                if not re.search(pattern2, tag):
                    clear_tag = " ".join([morph.normal_forms(token)[0] for token in tag.split()])
                    tags2.append(clear_tag)
                    if len(re.split(pattern5, clear_tag)):
                        for pod_tag in re.split(pattern5, clear_tag):
                            tags2.append(pod_tag)

    if len(tags) > 0:
        keywords = set(tags + tags2)
    else:
        keywords = set(tags1 + tags2)
    return keywords


for i in range(1, 21):
    f = open(f'/data/notebook_files/vacancies/{i}.txt')
    print(i)
    print(get_tags_vacancy(f.read()))
    print("---------")


1
{'numpy', 'бд такой', 'статистика математический', 'алгоритм обучение', 'гит', 'python пакет', 'вероятность', 'анализ', 'scipy', 'анализ данные', 'данные', 'постгрес', 'сцикит-леарн', 'сципы', 'pandas', 'писать код', 'вероятность статистика', 'код', 'пакет', 'пакет анализ', 'сцикитлеарн', 'обучение', 'docker', 'классический', 'алгоритм теория', 'пытхон', 'линуx', 'сqл', 'scikit-learn', 'статистика', 'scikitlearn', 'нумпы', 'python', 'доцкер', 'sql', 'структура', 'такой', 'обучение машинный', 'математический', 'пандас', 'писать', 'git', 'алгоритм', 'linux', 'бд', 'postgres', 'теория', 'алгоритм структура', 'алгоритм классический', 'машинный', 'теория вероятность'}
---------
2
{'язык запрос', 'фора', 'конфигурация типовой', 'запрос управляемый', 'управляемый', 'один', 'управляемый фора', 'типовой', 'язык', 'конфигурация', '1с', 'конфигурация один', 'запрос'}
---------
3
{'хотеть хотеть', 'развивать', 'желание создать', 'мышление аналитический', 'создать развивать', 'мышление', 'хотеть'

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_courses():
    url = 'https://gb.ru/courses/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = [i.get('href') for i in soup.find_all('a', class_='card_full_link')]
    name = [i.span.text.strip().replace("\xa0", " ") for i in soup.find_all('div', class_='direction-card__title')]
    decription = [i.text.strip().replace("\xa0", " ") for i in soup.find_all('div', class_='direction-card__text')]
    term = [i.span.text.strip().replace("\xa0", " ") for i in soup.find_all('div', class_='direction-card__info-text ui-text-body--6')]
    courses = [(name[i], links[i], decription[i], term[i]) for i in range(len(links))]
    
    return courses

def course_parsing(url):
    pattr = "\((.+)\)"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html')
    skils = set()
    for description in soup.find_all('div', class_="promo-tech__item gkb-promo__tag _large ui-text-body--5"):
        tmp = set(description.span.text.strip().lower().split('/'))
        for tag in tmp:
            if len(re.findall(pattr, tag)):
                tmp = tmp | set([i.strip() for j in re.findall(pattr, tag) for i in j.split('/')]) | \
                      set([i.strip() for j in re.split(pattr, tag) for i in j.split('/')])
        skils = skils | tmp
    
    ignore_skills = set(['и другие', ''])
    return skils - ignore_skills



In [13]:
course_skils = [(i[0], i[1], course_parsing(i[1]), i[2], i[3]) for i in get_courses()]

df_course_skils = pd.DataFrame(course_skils, columns=["name", "url", "tags", "decription", "term"])
without_tags = df_course_skils[df_course_skils["tags"] == set()]

another_tags = []
with open("/data/notebook_files/tags.txt") as file:
    s = file.read().replace("\n", "").split(";")
    #print(s)
    #print(len(s))
    for i in range(0, len(s), 2):
        url = s[i].strip()
        tags = s[i+1].strip().replace("{", "").replace("}", "").replace("'", "").split(",")
        for i, t in enumerate(tags):
            tags[i] = t.strip()
        
        another_tags.append((url, set(tags)))
df_another_tags = pd.DataFrame(another_tags, columns=["url", "tags"])

df_courses = pd.DataFrame(course_skils, columns=["name", "url", "tags", "decription", "term"])
df_courses["id"] = df_courses.index

df_all_courses = pd.merge(df_courses,df_another_tags, on='url', how='left')
df_all_courses.loc[df_all_courses["tags_x"] == set(), "tags_x"] = df_all_courses.loc[df_all_courses["tags_y"] != set(), "tags_y"]
df_all_courses = df_all_courses.drop(columns=["tags_y"]).dropna()
df_all_courses = df_all_courses.rename(columns={"tags_x": "tags"})
drop_id = []
for i in df_all_courses.values:
    if i[3] == "":
        drop_id = i[5]
df_all_courses = df_all_courses.drop(drop_id)

all_tags = set()
for i in course_skils:
    all_tags = all_tags | i[2]

def check(input_tags, all_tags, df_course_skils):
    # Находим курс, который покрыват больше всего

    courses = df_course_skils.copy(deep=True)
    courses["id"] = courses.index
    
    best_id = []
    coverage = []
    input_tags = input_tags & all_tags
    print(input_tags)
    input_tags_len = len(input_tags)
    indexes = set(courses.index)

    courses["coverage"] = [0]*len(indexes)
    while len(input_tags)!=0 and len(best_id)<=4:
        max_tags_len = -1
        max_tags_id = -1
        min_unnecessary_tags_len = -1

        for i in indexes:
            course = courses.loc[i]
            tags_len = len(input_tags & course[2])
            unnecessary_tags = len(course[2] - input_tags)
            if max_tags_len < tags_len or ((max_tags_len == tags_len) and (unnecessary_tags < min_unnecessary_tags_len)):
                max_tags_len = tags_len
                max_tags_id = i
                min_unnecessary_tags_len = unnecessary_tags
        best_id.append(max_tags_id)
        courses.loc[max_tags_id, "coverage"] = round(max_tags_len/input_tags_len*100)
    
        input_tags = input_tags - courses.loc[max_tags_id][2]
        indexes = indexes - set([max_tags_id])
    
    ans = courses[courses.index.isin (best_id)]
    ans = ans.drop(columns=["id"])
    
    return ans.sort_values("coverage", ascending=False)
    
check({"wi-fi", "cmake", "http", "udp", "stl"}, all_tags, df_all_courses)

{'http', 'cmake', 'udp', 'wi-fi', 'stl'}


,name,url,tags,decription,term,coverage
41,Разработчик на C++ с нуля до Junior,https://gb.ru/geek_university/developer/progra...,"{git, linux, qt, tcp, http, udp, stl, c++, cmake}",Углубленная теория и больше практики. в конце ...,12 месяцев,80
23,Разработчик умных устройств,https://gb.ru/geek_university/developer/progra...,"{iot, wi-fi, ble, c}","Научитесь проектировать, собирать и программир...",9 месяцев,20


In [79]:
url = 'https://gb.ru/geek_university/developer/product-manager-gb'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
navyk = [i.text.strip().replace("\xa0", " ").replace("  ", "\n") for i in soup.find_all('ul', class_='tools')]

nav = set()
for i in navyk:
    
    nav = nav | set([j.strip() for j in i.split("\n")])
ignore = set(["", 'и другие инструменты'])
nav = nav - ignore
nav

{'+ 41 инструмент',
 'CustDev',
 'Figma',
 'JTBD',
 'Lean Canvas',
 'Product Market Fit',
 'Retention',
 'User Story',
 'Ценностное предложение',
 'Юнит-экономика'}

,url,tags
0,https://gb.ru/s/ai-integration-specialist,"{Product Market Fit, User Story, Юнит-экономик..."
1,https://gb.ru/geek_university/developer/qa-eng...,"{RESTful API, JUnit, NoSQL, SQL, CSS, Pytest, ..."
2,https://gb.ru/geek_university/design/graphic-d...,"{Adobe Photoshop, Figma, Adobe Illustrator, Ad..."
3,https://gb.ru/geek_university/developer/progra...,"{JUnit, SQL, PostgreSQL, CSS, Git, XML, Java D..."
4,https://gb.ru/geek_university/developer/progra...,"{Hive, SQL, Matplotlib, Scikit-Learn, Git, Pyt..."
5,https://gb.ru/geek_university/developer/progra...,"{JavaScript, Vue, WebSocket, SessionStorage, B..."
6,https://gb.ru/geek_university/marketing/smm-ma...,"{TGStat, Flyvi, Главред, Telegram, VN, VK, Goo..."
7,https://gb.ru/geek_university/developer/qa-eng...,"{Python, JavaScript, JUnit, Allure, NoSQL, Jav..."
8,https://gb.ru/geek_university/developer/progra...,"{FastAPI, SQL, CSS, Git, Python, XML, Django, ..."
9,https://gb.ru/geek_university/developer/progra...,"{SQL, GitHub, PostgreSQL, Xcode, Git, Swift Pl..."


In [14]:
f = open("/data/notebook_files/vacancies/5.txt")
text = f.read()
tags = get_tags_vacancy(text)
check(tags, all_tags, df_course_skils)

{'cd', 'ospf', 'linux'}


,name,url,tags,decription,term,coverage
68,Сетевой инженер,https://gb.ru/geek_university/developer/archit...,"{stp, linux, ip, bgp, tcp, python, ospf, fhrp}",Вы с нуля научитесь настраивать и обслуживать ...,12 месяцев,67
19,DevOps-инженер: быстрый старт в профессии,https://gb.ru/geek_university/developer/archit...,"{ethernet, dns, nosql, kubernetes (k8s), unix,...","Вы изучите методы и инструменты DevOps, научит...",9 месяцев,33


In [23]:
from navec import Navec
import slovnet

navec = Navec.load("navec_news_v1_1B_250K_300d_100q.tar")
syntax = slovnet.Syntax.load("slovnet_syntax_news_v1.tar")

syntax. navec(navec)

Syntax(
    infer=SyntaxInfer(
        model=Syntax(
            emb=WordShapeEmbedding(
                word=NavecEmbedding(
                    id='news_v1_1B_250K_300d_100q',
                    indexes=Weight(
                        shape=[250002,
                         100],
                        dtype='uint8',
                        array=array([[176, 222, 248, ..., 244, 183, 191],
                               [215, 200, 168, ..., 120, 217,  21],
                               [ 83, 174,  54, ..., 106,  88, 251],
                               ...,
                               [133, 125, 123, ..., 124,  94,  24],
                               [183,  49, 180, ..., 151, 167,  68],
                               [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)
                    ),
                    codes=Weight(
                        shape=[100,
                         256,
                         3],
                        dtype='float32',
                    

In [38]:
sent = "Этим летом стартует новый сезон Paralect Startup Summer 2024 — курса стажировки для Full-Stack Developer. Инженеры из Paralect поделятся с тобой опытом, помогут стать частью реального проекта и справиться с трудностями на всех этапах: от старта до запуска в продакшн. В прошлом году мы приняли более 200 заявок, провели 70 интервью, 12 ребят взяли на курсы и 12 - трудоустроили в Paralect. А самая крутая новость — после прохождения курса у тебя будет возможность получить оффер и стать частью команды Paralect! Немного фактов о Paralect: Команда Paralect — это больше 150 специалистов высокого класса, которые работают над проектами в доменах: FinTech, Insurance, HealthCare, Marketing. Мы не ограничиваемся стандартным технологическим стеком: .Net, Node.js., MongoDB, Kafka, React/Vue, Docker, но и развиваем экспертизу в ML, AI, Blockchain. Что тебя ждет?  Участие в реальном стартапе под присмотром опытных менторов и лекторов; Около 20 теоретических лекций, 21 практическое занятие, 250 часов работы над реальным продуктом, с настоящим фаундером; Практика с такими технологиями, как React, PostCSS, Node.js, MongoDB, Koa, Socket.io, Redis, Git, Docker и др.; Крутые ивенты, тимбилдинги и, конечно же, приятный мерч, куча фоток и видео.  Что для этого необходимо?  Иметь возможность уделять на курсы от 8 часов с понедельника по пятницу; Базовые знания HTML, CSS; Знать основы JavaScript, React, HTTP и Node.js; Владеть английским на уровне Intermediate; Заполнить форму регистрации (её ты можешь получить после отклика); Выполнить тестовое задание, информацию о котором ты сможешь узнать при регистрации на курсы.  Ждем твоего отклика! FAQКогда начинаются курсы и сколько они длятся?  Мы принимаем заявки до 24 мая, затем до 24 июня проводим интервью. Сами курсы будут длиться 2 месяца (с июля по август). Курсы платные? Нет, курсы полностью бесплатные. Какой формат обучения? Курсы будут проходить оффлайн в минском офисе Paralect. Курсы full-time?Да, тебе нужно будет уделять по 8 часов с понедельника по пятницу."

doc = seg_text(sent)
doc

{'text': 'Этим летом стартует новый сезон Paralect Startup Summer 2024 — курса стажировки для Full-Stack Developer. Инженеры из Paralect поделятся с тобой опытом, помогут стать частью реального проекта и справиться с трудностями на всех этапах: от старта до запуска в продакшн. В прошлом году мы приняли более 200 заявок, провели 70 интервью, 12 ребят взяли на курсы и 12 - трудоустроили в Paralect. А самая крутая новость — после прохождения курса у тебя будет возможность получить оффер и стать частью команды Paralect! Немного фактов о Paralect: Команда Paralect — это больше 150 специалистов высокого класса, которые работают над проектами в доменах: FinTech, Insurance, HealthCare, Marketing. Мы не ограничиваемся стандартным технологическим стеком: .Net, Node.js., MongoDB, Kafka, React/Vue, Docker, но и развиваем экспертизу в ML, AI, Blockchain. Что тебя ждет?  Участие в реальном стартапе под присмотром опытных менторов и лекторов; Около 20 теоретических лекций, 21 практическое занятие, 25

In [39]:
syntax_markup = syntax(doc["tokens"][0])

for token_info in syntax_markup.tokens:
    print(token_info)

SyntaxToken(id='1', text='Этим', head_id='2', rel='det')
SyntaxToken(id='2', text='летом', head_id='3', rel='obl')
SyntaxToken(id='3', text='стартует', head_id='0', rel='root')
SyntaxToken(id='4', text='новый', head_id='5', rel='amod')
SyntaxToken(id='5', text='сезон', head_id='3', rel='nsubj')
SyntaxToken(id='6', text='Paralect', head_id='5', rel='appos')
SyntaxToken(id='7', text='Startup', head_id='6', rel='flat:foreign')
SyntaxToken(id='8', text='Summer', head_id='6', rel='flat:foreign')
SyntaxToken(id='9', text='2024', head_id='6', rel='flat:foreign')
SyntaxToken(id='10', text='—', head_id='11', rel='punct')
SyntaxToken(id='11', text='курса', head_id='5', rel='appos')
SyntaxToken(id='12', text='стажировки', head_id='11', rel='nmod')
SyntaxToken(id='13', text='для', head_id='14', rel='case')
SyntaxToken(id='14', text='Full-Stack', head_id='12', rel='nmod')
SyntaxToken(id='15', text='Developer', head_id='14', rel='flat:foreign')
SyntaxToken(id='16', text='.', head_id='3', rel='punct'

In [59]:
doc = seg_text(sent)
doc = extract_candidates(doc)
doc = syntax_collocations(doc)

print(doc["collocations"])

{'провели интервью', 'Владеть английским', 'ивенты мерч', 'новость прохождения', 'бесплатные бесплатные', 'форму регистрации', 'бесплатные курсы', '24 июня', 'ивенты тимбилдинги', 'специалистов работают', 'частью команды', 'понедельника пятницу', 'работы продуктом', 'приняли году', 'специалистов Команда', 'справиться трудностями', 'стеком технологическим', 'лекций теоретических', 'технологиями такими', 'развиваем экспертизу', 'ивенты куча', 'проходить Курсы', 'работают специалистов', 'офисе минском', 'работают которые', 'июля август', 'менторов лекторов', 'поделятся справиться', 'взяли ребят', 'Иметь возможность', 'информацию котором', 'курсы Сами', 'уделять понедельника', 'мерч приятный', 'получить оффер', 'Ждем отклика', 'возможность получить', 'стеком стандартным', 'Практика технологиями', 'Немного фактов', 'новость крутая', 'месяца июля', 'фаундером настоящим', 'FAQ начинаются', 'отклика твоего', 'длиться курсы', '24 мая', 'приняли заявок', 'поделятся Инженеры', 'регистрации можешь

In [27]:
# подключаем urlopen из модуля urllib
from urllib.request import urlopen

# подключаем библиотеку BeautifulSoup
from bs4 import BeautifulSoup

# получаем исходный код страницы
inner_html_code = str(urlopen('https://tyumen.hh.ru/vacancy/98065158?from=applicant_recommended&hhtmFrom=main').read(),'utf-8')

# отправляем исходный код страницы на обработку в библиотеку
inner_soup = BeautifulSoup(inner_html_code, "html.parser")

# выводим содержимое страницы
print(inner_soup.get_text())


Вакансия Помощник дизайнера в Тюмени, работа в компании ГРИФОНПроизошла ошибка. Попробуйте перезагрузить страницу.Для работы с нашим сайтом необходимо, чтобы Вы включили JavaScript в вашем браузере.ТюменьСоискателямРаботодателямГотовое резюмеКарьерная консультацияВсе сервисы ПомощьИщу работу ПоискСоздать резюмеВойтиВойтиНайтиПомощник дизайнераот 66 000 ₽ на рукиТребуемый опыт работы: не требуетсяПолная занятость, удаленная работаВозможна подработка: сменами по 4-6 часов или по вечерамОткликнутьсяООО ГРИФОН
Нет отзывовТюменьНапишите телефон, чтобы работодатель мог связаться с вамиПродолжитьЧтобы подтвердить, что вы не робот, введите текст с картинки: Другой текстEnglishНажимая «Продолжить», вы подтверждаете, что полностью принимаете условия Соглашения об оказании услуг по содействию в трудоустройстве (оферта) и ознакомились с политикой конфиденциальностиМы ищем человека (даже без опыта работы в этой сфере - научим сами), который будет готов разбираться в сфере дизайна. Нужны инициативн

In [48]:
get_tags_vacancy(inner_soup.get_text())

{'000',
 '1',
 '2024',
 '27',
 '27 апрель',
 '2х',
 '3',
 '46',
 '60',
 '66',
 '70',
 '80',
 'adobe',
 'adobe релакс',
 'after',
 'amo',
 'api',
 'apiпартнераминвесторамкаталог',
 'bitrix',
 'coreldraw',
 'crm',
 'effects',
 'english',
 'english©',
 'figma',
 'google',
 'headhunter',
 'headhunterheadhunter',
 'hh',
 'hhru',
 'hhruподписатьсяне',
 'illustrator',
 'javascript',
 'photoshop',
 'photoshopобработка',
 'pushуведомления',
 'ru',
 'switch',
 'to',
 'администрирование',
 'администрирование английский',
 'администрирование материал',
 'администрирование отзывпохожий',
 'администрирование персональный',
 'администрирование резюмеможный',
 'администрирование сайтетребование',
 'адобе',
 'айдентик',
 'амо',
 'анализ',
 'анастасия',
 'английский',
 'апи',
 'апипартнераминвесторамкаталог',
 'апрель',
 'ассистент',
 'афтер',
 'база',
 'база готовый',
 'база наш',
 'база оферта',
 'бизнес',
 'битриx',
 'браузер',
 'браузер ваш',
 'бренд',
 'брендинг',
 'бухгалтер',
 'ваш',
 'ввести',
 

In [ ]:
test_df = check({"wi-fi", "cmake", "http", "udp", "stl"}, all_tags, df_all_courses)
json_result = test_df.to_json()
json_result